In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
### Foursquare credentials
CLIENT_ID = 'VFSVWKKCSPYW21SIWA541D0GRC0N05AFEPSVF1UKVJGLHIDS' # your Foursquare ID
CLIENT_SECRET ='HOUCFKJGGAXADPFKA5VMDO4MHKRIVV1UZYSEDVP5X4SQEDYU'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
RADIUS = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VFSVWKKCSPYW21SIWA541D0GRC0N05AFEPSVF1UKVJGLHIDS
CLIENT_SECRET:HOUCFKJGGAXADPFKA5VMDO4MHKRIVV1UZYSEDVP5X4SQEDYU


### Prepare Toronto data

### Read the table from wikipedia

In [3]:
toronto=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [4]:
#select only the desired table
toronto=toronto[0]

### Remove Boroughs marked as "Not assigned"

In [5]:
toronto = toronto.loc[toronto['Borough']!='Not assigned']

### Case for not assigned neighborhoods 
(Note Not needed)

In [6]:
toronto['Neighborhood'] = np.where(toronto['Neighborhood']=='Not assigned', toronto['Borough'], toronto['Neighborhood'])

In [8]:
import geocoder # import geocoder
def get_lat_long(data):
    dict_Postal={}
    #initialize your variable to None
    lat_lng_coords = None
    for postal in list(data['Postal Code'].unique()):
            # loop until you get the coordinates
        while(lat_lng_coords is None):
          g = geocoder.google('{}, Toronto, Ontario'.format(postal))
          lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        dict_Postal[postal] = [latitude,longitude]
    return(dict_Postal)


In [9]:
geocodes= pd.read_csv('Geospatial_Coordinates.csv')

In [10]:
toronto = pd.merge(toronto,
             geocodes,
             how = 'left',
             on = 'Postal Code')

In [11]:
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Select Toronto Neighborhoods

In [13]:
toronto=toronto.loc[toronto['Borough'].str.contains('Toronto')]

## Prepare New York Data

In [31]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [32]:
neighborhoods_data = newyork_data['features']

In [33]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [34]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [35]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [36]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [37]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
def create_data_for_clustering(city):
    city_venues = getNearbyVenues(names=city['Neighborhood'],
                                   latitudes=city['Latitude'],
                                   longitudes=city['Longitude']
                                  )
        ### One Hot Encoding
    # one hot encoding
    city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

    # add neighborhood column back to dataframe
    city_onehot['Neighborhood'] = city_venues['Neighborhood'] 

    # move neighborhood column to the first column
    fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
    city_onehot = city_onehot[fixed_columns]
    city_grouped = city_onehot.groupby('Neighborhood').mean().reset_index()
    num_top_venues = 10

    indicators = ['st', 'nd', 'rd']

    # create columns according to number of top venues
    columns = ['Neighborhood']
    for ind in np.arange(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
        except:
            columns.append('{}th Most Common Venue'.format(ind+1))

    # create a new dataframe
    neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
    neighborhoods_venues_sorted['Neighborhood'] = city_grouped['Neighborhood']

    for ind in np.arange(city_grouped.shape[0]):
        neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_grouped.iloc[ind, :], num_top_venues)
    return(neighborhoods_venues_sorted,city_grouped)

## Use Foursquare API to get information about Toronto and New York

In [44]:
manhattan_venues_sorted, manhattan_grouped = create_data_for_clustering(manhattan_data)

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [45]:
toronto_venues_sorted, toronto_grouped = create_data_for_clustering(toronto)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

### Merge Toronto and New York Data

In [168]:
manhattan_grouped['City'] = 'Manhattan'
toronto_grouped['City'] = 'Toronto'
manhattan_venues_sorted['City'] = 'Manhattan'
toronto_venues_sorted['City'] = 'Toronto'
ClusterData = pd.concat([manhattan_grouped,toronto_grouped], axis = 0)
Venues_Sorted = pd.concat([manhattan_venues_sorted,toronto_venues_sorted], axis = 0)

In [169]:
ClusterData = ClusterData.fillna(0)

# DEFINE THE CONTROL NEIGHBORHOOD

In [170]:
def control_Neighborhood(neighborhood, latitude, longitude):
    ControlNeighborhood = pd.DataFrame({'Neighborhood' : [neighborhood],
                                       'Latitude': [latitude],
                                       'Longitude' : [longitude]})
    return(ControlNeighborhood)

## 1st Example: Saint-Germain, Paris

In [171]:
ControlNeighborhood =control_Neighborhood('Saint-Germain', 48.898899, 2.0938)

In [172]:
ControlNeighborhood

,Neighborhood,Latitude,Longitude
0,Saint-Germain,48.898899,2.0938


In [173]:
control_venues_sorted, control_grouped = create_data_for_clustering(ControlNeighborhood)

Saint-Germain


## Find common columns between train and test dataset

In [176]:
common_columns = ClusterData.columns & control_grouped.columns

In [177]:
from sklearn.cluster import KMeans




# set number of clusters
kclusters = 10


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ClusterData[common_columns].drop('Neighborhood', 1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
Venues_Sorted['Cluster Labels']= kmeans.labels_

In [178]:
Result_Cluster = int(kmeans.predict(control_grouped[common_columns].drop('Neighborhood', 1)))

In [180]:
print('If you liked {} you will like also {} in Manhattan'.format(list(ControlNeighborhood['Neighborhood']),
                                                               str(list(Venues_Sorted.loc[(Venues_Sorted['Cluster Labels']==Result_Cluster) & (Venues_Sorted['City']=='Manhattan') ]['Neighborhood']))   ))

If you liked ['Saint-Germain'] you will like also ['Chelsea', 'Chinatown', 'Civic Center', 'Clinton', 'East Village', 'Financial District', 'Gramercy', 'Greenwich Village', 'Hudson Yards', 'Lenox Hill', 'Lincoln Square', 'Little Italy', 'Manhattan Valley', 'Manhattanville', 'Marble Hill', 'Midtown', 'Midtown South', 'Murray Hill', 'Noho', 'Soho', 'Sutton Place', 'Tribeca', 'Turtle Bay', 'Upper East Side', 'Upper West Side', 'West Village', 'Yorkville'] in Manhattan


In [182]:
print('If you liked {} you will like also {} in Toronto'.format(list(ControlNeighborhood['Neighborhood']),
                                                               str(list(Venues_Sorted.loc[(Venues_Sorted['Cluster Labels']==Result_Cluster) & (Venues_Sorted['City']=='Toronto') ]['Neighborhood']))   ))

If you liked ['Saint-Germain'] you will like also ['CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport', 'Davisville', 'Garden District, Ryerson', 'Parkdale, Roncesvalles', 'Roselawn', 'The Danforth West, Riverdale'] in Toronto


### Second Example : Camden, London

In [183]:
ControlNeighborhood =control_Neighborhood('Camden',  51.536388, -0.140556)

ControlNeighborhood



,Neighborhood,Latitude,Longitude
0,Camden,51.536388,-0.140556


In [184]:
control_venues_sorted, control_grouped = create_data_for_clustering(ControlNeighborhood)

## Find common columns between train and test dataset

common_columns = ClusterData.columns & control_grouped.columns



Camden


In [185]:
common_columns


Index(['Neighborhood', 'Argentinian Restaurant', 'Bakery', 'Bar', 'Beer Bar',
       'Bookstore', 'Burger Joint', 'Bus Stop', 'Clothing Store',
       'Cocktail Bar', 'Coffee Shop', 'Convenience Store', 'Deli / Bodega',
       'Discount Store', 'Falafel Restaurant', 'Fast Food Restaurant',
       'Food Truck', 'Garden', 'Greek Restaurant', 'Grocery Store', 'Hostel',
       'Hotel', 'Ice Cream Shop', 'Indian Restaurant', 'Indie Theater',
       'Italian Restaurant', 'Japanese Restaurant', 'Jazz Club',
       'Kebab Restaurant', 'Lounge', 'Market', 'Mediterranean Restaurant',
       'Mexican Restaurant', 'Middle Eastern Restaurant', 'Music Venue',
       'Nightclub', 'Park', 'Pharmacy', 'Pizza Place', 'Pub',
       'Ramen Restaurant', 'Record Shop', 'Restaurant', 'Rock Club',
       'Sandwich Place', 'Shoe Store', 'Snack Place', 'Supermarket',
       'Sushi Restaurant', 'Tapas Restaurant', 'Tea Room', 'Thai Restaurant',
       'Turkish Restaurant', 'Vegetarian / Vegan Restaurant',
      

In [186]:
from sklearn.cluster import KMeans




# set number of clusters
kclusters = 10


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ClusterData[common_columns].drop('Neighborhood', 1))

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
Venues_Sorted['Cluster Labels']= kmeans.labels_

In [188]:
Result_Cluster = int(kmeans.predict(control_grouped[common_columns].drop('Neighborhood', 1)))

print('If you liked {} you will also like {} in Manhattan'.format(list(ControlNeighborhood['Neighborhood']),
                                                               str(list(Venues_Sorted.loc[(Venues_Sorted['Cluster Labels']==Result_Cluster) & (Venues_Sorted['City']=='Manhattan') ]['Neighborhood']))   ))



If you liked ['Camden'] you will like also ['Carnegie Hill', 'Chelsea', 'Civic Center', 'Manhattan Valley', 'Manhattanville'] in Manhattan


In [189]:
print('If you liked {} you will also like {} in Toronto'.format(list(ControlNeighborhood['Neighborhood']),
                                                               str(list(Venues_Sorted.loc[(Venues_Sorted['Cluster Labels']==Result_Cluster) & (Venues_Sorted['City']=='Toronto') ]['Neighborhood']))   ))

If you liked ['Camden'] you will like also ['Berczy Park', 'Brockton, Parkdale Village, Exhibition Place', 'Central Bay Street', 'Church and Wellesley', 'Commerce Court, Victoria Hotel', 'First Canadian Place, Underground city', 'Garden District, Ryerson', 'Harbourfront East, Union Station, Toronto Islands', 'North Toronto West, Lawrence Park', 'Parkdale, Roncesvalles', "Queen's Park, Ontario Provincial Government", 'Regent Park, Harbourfront', 'Richmond, Adelaide, King', 'St. James Town', 'St. James Town, Cabbagetown', 'Stn A PO Boxes', 'Studio District', 'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park', 'Toronto Dominion Centre, Design Exchange'] in Toronto


In [190]:
control_grouped

,Neighborhood,Argentinian Restaurant,Bakery,Bar,Beer Bar,Bookstore,Burger Joint,Bus Stop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Discount Store,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Food Truck,Fruit & Vegetable Store,Garden,Greek Restaurant,Grocery Store,Halal Restaurant,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Kebab Restaurant,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Music Venue,Nightclub,Park,Pharmacy,Pizza Place,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Sandwich Place,Shoe Store,Snack Place,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Camden,0.01,0.01,0.03,0.01,0.01,0.03,0.01,0.01,0.02,0.08,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.05,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.04,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.03,0.01,0.02,0.02,0.04,0.01,0.06,0.01,0.02,0.02,0.02,0.02,0.02,0.01,0.03,0.02,0.01,0.02,0.01,0.01,0.01
